CS524: Introduction to Optimization <br>Lecture 16: Write csv and excel
======================================
## by Michael C. Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison
## October 11, 2023

In [1]:
%load_ext gams.magic
m = gams.exchange_container

In [2]:
%%gams
$Title  A Transportation Problem (TRNSPORT,SEQ=1)
$Ontext

This problem finds a least cost shipping schedule that meets
requirements at markets and supplies at factories.


Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions. 
Princeton University Press, Princeton, New Jersey, 1963.

This formulation is described in detail in:
Rosenthal, R E, Chapter 2: A GAMS Tutorial. In GAMS: A User's Guide. 
The Scientific Press, Redwood City, California, 1988.

The line numbers will not match those in the book because of these 
comments.

$Offtext


  Sets
       i   canning plants   / seattle, san-diego /
       j   markets          / new-york, chicago, topeka / ;

  Parameters

       a(i)  capacity of plant i in cases
         /    seattle     350
              san-diego   600  /

       b(j)  demand at market j in cases
         /    new-york    325
              chicago     300
              topeka      275  / ;

  Table d(i,j)  distance in thousands of miles
                    new-york       chicago      topeka
      seattle          2.5           1.7          1.8
      san-diego        2.5           1.8          1.4  ;

  Scalar f  freight in dollars per case per thousand miles  /90/ ;

  Parameter c(i,j)  transport cost in thousands of dollars per case ;

            c(i,j) = f * d(i,j) / 1000 ;

  Variables
       x(i,j)  shipment quantities in cases
       z       total transportation costs in thousands of dollars ;

  Positive Variable x ;

  Equations
       cost        define objective function
       supply(i)   observe supply limit at plant i
       demand(j)   satisfy demand at market j ;

  cost ..        z  =e=  sum((i,j), c(i,j)*x(i,j)) ;

  supply(i) ..   sum(j, x(i,j))  =l=  a(i) ;

  demand(j) ..   sum(i, x(i,j))  =g=  b(j) ;

  Model transport /all/ ;

Solve transport using lp minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),153.675,6,7,LP,CPLEX,0.004


## CSV output via put statements (execution time)

In [3]:
%%gams

file results / trnsport_outG.csv /;
put results;
results.pc = 2;
put results "i,j,level,marginal,lower,upper,scale"/;
results.pc = 5;
loop((i,j), put i.tl, j.tl, x.l(i,j), x.m(i,j), x.lo(i,j), x.up(i,j), x.scale(i,j) / );
putclose;

# CSV output using pandas

In [4]:
# Create a Pandas dataframe from some GAMS data (x variable).
x = m.data['x'].records

# Export to csv file
x.to_csv('trnsport_outP.csv', index = False)

# CSV output via Connect (execution time)

In [5]:
%%gams

* Only level values of x exported

EmbeddedCode Connect:
- GAMSReader:
    symbols: [ {name: c}, {name: x} ]
- Projection:
    name: x.l(i,j)
    newName: x_level(i,j)
- CSVWriter:
    file: trnsport_outC.csv
    name: c
    unstack: False
- CSVWriter:
    file: shipment_quantities.csv
    name: x_level
    unstack: True
endEmbeddedCode


## CSV output via Connect and gdx file (execution time)

In [6]:
%%gams

# Can do this via gdx file and GDXReader
execute_unload "trnsport_output", i,j,a,b,c,d,f,x,z,cost,supply,demand;

EmbeddedCode Connect:
- GDXReader:
    file: trnsport_output.gdx
    symbols: [ {name: c}, {name: x} ]
- Projection:
    name: x.l(i,j)
    newName: x_level(i,j)
- CSVWriter:
    file: trnsport_c.csv
    name: c
    unstack: False
- CSVWriter:
    file: trnsport_outC.csv
    name: x_level
    unstack: True
endEmbeddedCode

In [7]:
# could also do in python via connectDB
from gams.connect import ConnectDatabase
cdb = ConnectDatabase(gams._system_directory, gams)
cdb.exec_task({'GDXReader': {'file': 'trnsport_output.gdx', 'symbols': [{'name': 'x'}]}})
cdb.exec_task({'Projection': {'name': 'x.l(i,j)', 'newName': 'x_level(i,j)'}})
cdb.exec_task({'CSVWriter': {'file': 'trnsport_outC.csv', 'name': 'x_level', 'unstack': True}})

# Excel output via pandas writer

In [8]:
import pandas as pd

# Create a Pandas dataframe from some GAMS data (x variable).
x = m.data['x'].records
d = m.data['d'].records

# Create a Pandas Excel writer
excel_file = 'trnsport_output.xlsx'
writer = pd.ExcelWriter(excel_file)

# Convert the dataframe to an XlsxWriter Excel object.
# Can write multiple sheets using this routine
x.to_excel(writer, sheet_name='Sheet1')
d.to_excel(writer, sheet_name='Sheet2')

# Close the Pandas Excel writer and output the Excel file.
writer.close()

In [9]:
%gams_cleanup --closedown